In [1]:
%load_ext autoreload
%autoreload 2
import urllib3, json, certifi
import HTMLx
from IPython.display import HTML, display
from operator import itemgetter 

In [2]:
MITREFILE = "enterprise-attack.json"
MITREURL = "https://raw.githubusercontent.com/mitre/cti/master/enterprise-attack/enterprise-attack.json"

In [3]:
http = urllib3.PoolManager(cert_reqs='CERT_REQUIRED',ca_certs=certifi.where())
chunk_size = 1024
r = http.request('GET', MITREURL, preload_content=False)

with open(MITREFILE, 'wb') as out:
    while True:
        data = r.read(chunk_size)
        if not data:
            break
        out.write(data)

r.release_conn()

In [4]:
with open(MITREFILE) as f:
    m = json.load(f)

In [5]:
print( len(m["objects"]))

4548


# Number of objects of different type

In [6]:
res = dict()
for o in m["objects"]:
    #t = o["id"][0:o["id"].find("--")]
    t = o["type"]
    if t == "relationship":
        s = o["source_ref"][0:o["source_ref"].find("--")]
        d = o["target_ref"][0:o["target_ref"].find("--")]
        t += ":"+s+"->"+o["relationship_type"]+"->"+d
    res[t] = res.get(t,0) + 1

display(HTML(HTMLx.html_table(["Object","Cnt"],map(lambda x: [x, res[x]],res.keys()))))
with open("objects.html","w") as f:
    f.write(HTMLx.html_table(["Object","Cnt"],map(lambda x: [x, res[x]],res.keys())))


In [7]:
platform = "Windows"
display(HTML("<H1>{}</H1>".format(platform)))

## APT actors uses Techniques

In [8]:
res = dict()

for o in m["objects"]:
    if o["type"] == "attack-pattern" and len(list(filter(lambda x: x == platform, o["x_mitre_platforms"]))) > 0:
        
        mitre_t = list(filter(lambda x: x["source_name"] == "mitre-attack", o["external_references"]))[0]["external_id"]
        k = mitre_t + ": " + o["name"]             
        res[k] = dict()
        res[k]["x_mitre_data_sources"] = list()
        res[k]["x_mitre_data_sources"] = o.get("x_mitre_data_sources",[])
          
        for c in ["intrusion-set","malware","tool"]: #APT, #malware, #tools
            res[k][c] = 0
        
        for o2 in m["objects"]: 
            if o2["type"] == "relationship" \
            and o2["relationship_type"] == "uses" \
            and o2["target_ref"] == o["id"]:
                k2 = o2["source_ref"][0:o2["source_ref"].find("--")]
                if k2 in ["intrusion-set","malware","tool"]: 
                    res[k][k2] = res[k].get(k2,0) + 1


data = list()
for kk, aa, mm, tt, ds in sorted ( ( (k, res[k]["intrusion-set"], res[k]["malware"], res[k]["tool"],res[k]["x_mitre_data_sources"]) for k in res.keys()), \
                          key = itemgetter(1), reverse = True ) :
    data.append([kk, aa, mm, tt, "<BR>".join(ds)])

display(HTML(HTMLx.html_table(["techique", "intrusion-set","malware","tool","data_sources"],data)))

## Telemetry required for Techniques

In [9]:
res = dict()
no_data_sources = list()
for o in m["objects"]:
    t = o["type"]
    if t == "attack-pattern" and len(list(filter(lambda x: x == platform, o["x_mitre_platforms"]))) > 0:
        if o.get("x_mitre_data_sources") is None:
            no_data_sources.append(o["name"])
        else:
            for s in o["x_mitre_data_sources"]:
                res[s] = res.get(s,0) + 1

display(HTML(
    HTMLx.html_table(["data_sources", "attack-pattern"],\
                     list([key,value] for key, value in sorted(res.items(), key = itemgetter(1), reverse = True)))
))

## Windows techniques without required telemetry specified

In [10]:
for i in no_data_sources:
    print(i)

Peripheral Device Discovery
